<img src="./University_Debrecen_logo.jpg" alt="Drawing" style="width: 200px;"/>

# Generate E_Figure 1 of the SIMV-VG-PS paper

### 1. Import the required modules and set options

In [1]:
import sys
import os
import datetime

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from pandas import DataFrame
from matplotlib import dates

# module containing function to import and process data
import fast_data_analyser_debrecen

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

# suppress chained assigment warning
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
print(f'Python version: {sys.version}')
print(f'pandas version: {pd.__version__}')
print(f'matplotlib version: {mpl.__version__}')
print(f'numPy version: {np.__version__}')

Python version: 3.9.12 (main, Apr  5 2022, 01:53:17) 
[Clang 12.0.0 ]
pandas version: 1.4.4
matplotlib version: 3.6.2
numPy version: 1.21.5


In [ ]:
# Topic of the Notebook which will also be the name of the subfolder containing results
TOPIC = 'SIMV_VG_PS'

# Directory containing clinical and blood gas data
DIR_READ_CLIN = os.path.join(os.sep, 'Users', 'guszti', 'ventilation_draeger_debrecen')

# Folder on external drive to read the ventilation data from
DIR_READ_VENT =  os.path.join(os.sep, 'Volumes', 'Guszti', 'draeger_debrecen',)

# Folder on external drive to read the ventilation data from
DIR_READ_VENT_2 =  os.path.join(os.sep, 'Elements_new', 'Ventilator_data', 'Draeger_Debrecen',)

# Folder to write statistics and reports on the group
DIR_WRITE = os.path.join(os.sep, 'Users', 'guszti', 'ventilation_draeger_debrecen', 'Analyses', TOPIC)
os.makedirs(DIR_WRITE, exist_ok = True)

# Folder on external drive to export graphs and data about individual recordings
DATA_DUMP = os.path.join(os.sep, 'Volumes', 'Guszti', 'data_dump', 'draeger_debrecen', TOPIC)
os.makedirs(DATA_DUMP, exist_ok = True)

In [ ]:
DIR_READ_CLIN, DIR_READ_VENT, DIR_READ_VENT_2, DIR_WRITE, DATA_DUMP

In [ ]:
# Select the patient to analyse
recording = 'LVD016'

### 2. Import data

In [ ]:
clinical_details_recordings = pd.read_csv(os.path.join(DIR_READ_CLIN, 'clinical_details_recordings.csv'))
clinical_details_recordings = clinical_details_recordings.set_index(['Patient', 'Recording'])
clinical_details_recordings;

In [ ]:
clinical_details_recordings.loc[recording]

In [ ]:
%%time

fast_data = fast_data_analyser_debrecen.process_fast_data(recording)

In [ ]:
%%time

predicted_breaths = fast_data_analyser_debrecen.process_predicted_breaths(recording)

In [ ]:
list(fast_data.keys())

### 3. Select the part recording which contains the data to be visualize

In [ ]:
# Choose a part of the recoridng
part = sorted(fast_data.keys())[3]
print(part)

##### Choose a period of the this recording to study further

This cannot be too large. Processing time will be approximately 1 minute / 1000 breaths. As approximatelly 1 breath happens every second, this corresponds to ~15 minutes of recording. Therefore, it is not really feasible to choose a period longer than 1 hour unless you want to run the program for a long time. 

Choose the start and end time in `2019-03-20 12:34:18` format

In [ ]:
fast_data[part].head()

In [ ]:
fast_data[part].tail()

In [ ]:
weight = clinical_details_recordings.loc[recording, part]['Current weight'].values[0] / 1000
weight

In [ ]:
start_time = '2022-01-25 12:40:00'
end_time =   '2022-01-25 12:50:00'

In [ ]:
start_time = pd.Timestamp(start_time)
end_time =   pd.Timestamp(end_time)

start_data_point = start_time - fast_data[part].index[0]
end_data_point = end_time - fast_data[part].index[0]
end_data_point - start_data_point 

In [ ]:
start_time, end_time

In [ ]:
a = predicted_breaths[part]
                                 
period_1 = a[(a['breath_start'] > int(start_data_point.total_seconds() * 100)) & 
           (a['breath_start'] < int(end_data_point.total_seconds() * 100) )]

print('Number of breaths: %s' % len(period_1))

In [ ]:
start = period_1.index[0]
end = period_1.index[-1]

br_selected = list(range(start, end+1))
tag = '%d_%d' % (start, end)

In [ ]:
breaths = {}
for br in br_selected:
    breaths[br] = fast_data[part][predicted_breaths[part].iloc[br,1] : predicted_breaths[part].iloc[br,2]]

The next code will take  `approximately 1 minute / 1500 breaths`

In [ ]:
%%time
breaths_data = fast_data_analyser_debrecen.flow_integrater(breaths)

In [ ]:
breaths_all = pd.concat(breaths_data)

In [ ]:
# number of inflations
breaths_all.index[-1][0] - breaths_all.index[0][0]

### 4. Visualize multiple inflations as waveforms and loops

In [ ]:
print(recording, part, tag)

In [ ]:
start_seq = start+60; end_seq = start+80
brs_range = np.arange(start_seq, end_seq+1)

In [ ]:
breaths_selected = {key: value for key, value in breaths.items() if key in brs_range}
breaths_selected = pd.concat(breaths_selected, sort = True)
breaths_selected = breaths_selected.reset_index(level = 0)

In [ ]:
recording

In [ ]:
fast_data_analyser_debrecen.waves_multiple(recording=recording, part=part, weight=weight,
                                           breaths=breaths_selected, write = True, folder = DATA_DUMP)

In [ ]:
filetype = 'pdf'
dpi = 300

fig, axes = plt.subplots(nrows=3, ncols=1, sharex = 'all', sharey = 'none')
fig.set_size_inches(12,12); fig.set_label('res')
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.1)
    
breaths_selected.pressure.plot(ax = axes[0], color = 'red', title = 'Pressure', linewidth=2, 
    label = 'Pressure',ylim = [0, (breaths_selected.pressure.max() * 1.5)]);
breaths_selected.flow.plot(ax = axes[1], color = 'green', title = 'Flow', linewidth=2,
    label = 'Flow', ylim = [(breaths_selected.flow.min() * 1.2), (breaths_selected.flow.max() * 1.5)])
vol_corr = breaths_selected.volume / weight
vol_corr.plot(ax = axes[2], color = 'blue', title = 'Volume', linewidth=2,
    label = 'Volume', ylim = [0, (vol_corr.max() * 1.5)], x_compat=True)
xmin, xmax = axes[1].get_xlim()
axes[1].hlines(0, xmin, xmax, color = 'black', linewidth = 2)
    
majorFmt = dates.DateFormatter('%H:%M:%S')  
axes[2].xaxis.set_major_formatter(majorFmt)
plt.setp(axes[2].xaxis.get_majorticklabels(), rotation=0, fontsize = 14, 
             horizontalalignment = 'center')
      
axes[0].set_xlabel(''); axes[1].set_xlabel('')
axes[2].set_xlabel('Time', size = 14, color = 'black', rotation = 0 )
axes[0].set_ylabel('mbar', size = 14, color = 'black')
axes[1].set_ylabel('L/min', size = 14, color = 'black')
axes[2].set_ylabel('mL/kg', size = 14, color = 'black')
axes[0].legend(); axes[1].legend() ; axes[2].legend() 
axes[0].set_title(''); axes[1].set_title(''); axes[2].set_title('')
axes[0].grid('on', linestyle='-', linewidth=0.5, color = 'gray') 
axes[1].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
axes[2].grid('on', linestyle='-', linewidth=0.5, color = 'gray')

### 5. Figure 1

#### Figure 1A

In [ ]:
start_seq = start+30; end_seq = start+90
brs_range = np.arange(start_seq, end_seq+1)

breaths_selected_1 = {key: value for key, value in breaths.items() if key in brs_range}
breaths_selected_1 = pd.concat(breaths_selected_1, sort = True)
breaths_selected_1 = breaths_selected_1.reset_index(level = 0)

filetype = 'pdf'
dpi = 300

fig, ax = plt.subplots(nrows=1, ncols=1)
fig.set_size_inches(12,4); fig.set_label('res')
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.1)

breaths_selected_1.pressure.plot(ax = ax, color = 'red', title = 'Pressure', linewidth=2, 
    label = 'Pressure',ylim = [0, 17]);

xmin, xmax = ax.get_xlim()
ax.hlines(10, xmin, xmax, color = 'black', linewidth = 2, linestyle = 'dashed', label = 'PEEP + PS')

ax.vlines(pd.to_datetime('2022-01-25 13:05:33.850'), 0,17, color = 'black', linewidth = 2, 
                         linestyle = 'dotted')
ax.vlines(pd.to_datetime('2022-01-25 13:05:49.250'), 0,17, color = 'black', linewidth = 2, 
                         linestyle = 'dotted',)

majorFmt = dates.DateFormatter('%H:%M:%S')  
ax.xaxis.set_major_formatter(majorFmt)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 14, horizontalalignment = 'center')
      
ax.set_xlabel('Time', size = 14, color = 'black', rotation = 0 )
ax.set_ylabel('mbar', size = 14, color = 'black')

ax.legend(loc=4, ncol=2)
ax.set_title(''); 
ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray') 
    
fig.savefig(os.path.join(DIR_WRITE, f'Figure_1A.{filetype}'), dpi = dpi, format = filetype, 
            bbox_inches='tight', pad_inches=0.1,)       

#### Figure 1B

In [ ]:
start_seq = start+60; end_seq = start+80
brs_range = np.arange(start_seq, end_seq+1)

breaths_selected_2 = {key: value for key, value in breaths.items() if key in brs_range}
breaths_selected_2 = pd.concat(breaths_selected_2, sort = True)
breaths_selected_2 = breaths_selected_2.reset_index(level = 0)

filetype = 'pdf'
dpi = 300

fig, axes = plt.subplots(nrows=3, ncols=1, sharex = 'all', sharey = 'none')
fig.set_size_inches(12,12); fig.set_label('res')
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.1)

breaths_selected_2.pressure.plot(ax = axes[0], color = 'red', title = 'Pressure', linewidth=2, 
    label = 'Pressure',ylim = [0, 17]);
breaths_selected_2.flow.plot(ax = axes[1], color = 'green', title = 'Flow', linewidth=2,
    label = 'Flow', ylim = [-3, 4])
vol_corr = breaths_selected_2.volume / weight
vol_corr.plot(ax = axes[2], color = 'blue', title = 'Volume', linewidth=2,
    label = 'Volume', ylim = [0, 9], x_compat=True)

axes[0].text(pd.to_datetime('2022-01-25 13:05:34.200'), 14, '*', fontsize = 18)
axes[0].text(pd.to_datetime('2022-01-25 13:05:36.170'), 14, '*', fontsize = 18)
axes[0].text(pd.to_datetime('2022-01-25 13:05:38.330'), 14, '*', fontsize = 18)
axes[0].text(pd.to_datetime('2022-01-25 13:05:39.860'), 14, '*', fontsize = 18)
axes[0].text(pd.to_datetime('2022-01-25 13:05:41.300'), 14, '*', fontsize = 18)
axes[0].text(pd.to_datetime('2022-01-25 13:05:43.400'), 14, '*', fontsize = 18)
axes[0].text(pd.to_datetime('2022-01-25 13:05:45.000'), 14, '*', fontsize = 18)
axes[0].text(pd.to_datetime('2022-01-25 13:05:46.400'), 14, '*', fontsize = 18)

xmin, xmax = axes[0].get_xlim()
axes[0].hlines(10, xmin, xmax, color = 'black', linewidth = 2, linestyle = 'dashed', label = 'PEEP + PS')
axes[1].hlines(0, xmin, xmax, color = 'black', linewidth = 2)
axes[2].hlines(4.7, xmin, xmax, color = 'black', linewidth = 2, linestyle = 'dashed', label = 'VTset')
   
majorFmt = dates.DateFormatter('%H:%M:%S')  
axes[2].xaxis.set_major_formatter(majorFmt)
plt.setp(axes[2].xaxis.get_majorticklabels(), rotation=0, fontsize = 14, horizontalalignment = 'center')
      
axes[0].set_xlabel(''); axes[1].set_xlabel('')
axes[2].set_xlabel('Time', size = 14, color = 'black', rotation = 0 )
axes[0].set_ylabel('mbar', size = 14, color = 'black')
axes[1].set_ylabel('L/min', size = 14, color = 'black')
axes[2].set_ylabel('mL/kg', size = 14, color = 'black')
axes[0].legend(loc=4, ncol=2); axes[1].legend() ; axes[2].legend(ncol=2)
axes[0].set_title(''); axes[1].set_title(''); axes[2].set_title('')
axes[0].grid('on', linestyle='-', linewidth=0.5, color = 'gray') 
axes[1].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
axes[2].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
fig.savefig(os.path.join(DIR_WRITE, f'Figure_1B.{filetype}'), dpi = dpi, format = filetype, 
            bbox_inches='tight', pad_inches=0.1,)       

#### Figure 1 combined (old version, colored)

In [ ]:
filetype = 'pdf'
dpi = 300

fig, axes = plt.subplots(nrows=4, ncols=1, sharex = 'none', sharey = 'none')
fig.set_size_inches(12,16); fig.set_label('res')
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.15)


# Figure 1A

start_seq = start+30; end_seq = start+90
brs_range = np.arange(start_seq, end_seq+1)
breaths_selected_1 = {key: value for key, value in breaths.items() if key in brs_range}
breaths_selected_1 = pd.concat(breaths_selected_1, sort = True)
breaths_selected_1 = breaths_selected_1.reset_index(level = 0)

breaths_selected_1.pressure.plot(ax = axes[0], color = 'red', title = 'Pressure', linewidth=2, 
    label = 'Pressure',ylim = [0, 17]);
xmin, xmax = axes[0].get_xlim()
axes[0].hlines(10, xmin, xmax, color = 'black', linewidth = 2, linestyle = 'dashed', label = 'PEEP + PS')
axes[0].axvspan(pd.to_datetime('2022-01-25 13:05:33.850'), pd.to_datetime('2022-01-25 13:05:49.250'), 
                                                                     color='gray', alpha=0.4, lw=0)
majorFmt = dates.DateFormatter('%H:%M:%S')  
axes[0].xaxis.set_major_formatter(majorFmt)
plt.setp(axes[0].xaxis.get_majorticklabels(), rotation=0, fontsize = 14, horizontalalignment = 'center')
      
axes[0].set_xlabel('', size = 14, color = 'black', rotation = 0 )
axes[0].set_ylabel('mbar', size = 14, color = 'black')
axes[0].legend(loc=4, ncol=2)
axes[0].set_title('');
axes[0].grid('on', linestyle='-', linewidth=0.5, color = 'gray') 


# Figure 1B-C-D

start_seq = start+60; end_seq = start+80
brs_range = np.arange(start_seq, end_seq+1)
breaths_selected_2 = {key: value for key, value in breaths.items() if key in brs_range}
breaths_selected_2 = pd.concat(breaths_selected_2, sort = True)
breaths_selected_2 = breaths_selected_2.reset_index(level = 0)

breaths_selected_2.pressure.plot(ax = axes[1], color = 'red', title = 'Pressure', linewidth=2, 
    label = 'Pressure',ylim = [0, 17]);
breaths_selected_2.flow.plot(ax = axes[2], color = 'green', title = 'Flow', linewidth=2,
    label = 'Flow', ylim = [-3, 4])
vol_corr = breaths_selected_2.volume / weight
vol_corr.plot(ax = axes[3], color = 'blue', title = 'Volume', linewidth=2,
    label = 'Volume', ylim = [0, 9], x_compat=True)

majorFmt = dates.DateFormatter('%H:%M:%S')  
axes[3].xaxis.set_major_formatter(majorFmt)
plt.setp(axes[3].xaxis.get_majorticklabels(), rotation=0, fontsize = 14, horizontalalignment = 'center')

axes[1].text(pd.to_datetime('2022-01-25 13:05:34.200'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:36.170'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:38.330'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:39.860'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:41.300'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:43.400'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:45.000'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:46.400'), 14, '*', fontsize = 18)

xmin, xmax = axes[1].get_xlim()
axes[1].hlines(10, xmin, xmax, color = 'black', linewidth = 2, linestyle = 'dashed', label = 'PEEP + PS')
axes[2].hlines(0, xmin, xmax, color = 'black', linewidth = 2)
axes[3].hlines(4.7, xmin, xmax, color = 'black', linewidth = 2, linestyle = 'dashed', label = 'VTset')
   
axes[1].tick_params(bottom=False, labelbottom=False) 
axes[2].tick_params(bottom=False, labelbottom=False) 
    
axes[1].set_xlabel(''); axes[2].set_xlabel(''); axes[3].set_xlabel('Time', size = 14, color = 'black', rotation = 0 )
axes[1].set_ylabel('mbar', size = 14, color = 'black')
axes[2].set_ylabel('L/min', size = 14, color = 'black')
axes[3].set_ylabel('mL/kg', size = 14, color = 'black')
axes[1].legend(loc=4, ncol=2); axes[1].legend() ; axes[2].legend(ncol=2); axes[3].legend(ncol=2); 
axes[0].set_title(''); axes[1].set_title(''); axes[2].set_title(''); axes[3].set_title('')

fig.text(0.06, 0.90, 'A', fontsize = 16); fig.text(0.06, 0.72, 'B', fontsize = 16)
fig.text(0.06, 0.54, 'C', fontsize = 16); fig.text(0.06, 0.36, 'D', fontsize = 16)
    
fig.savefig(os.path.join(DIR_WRITE, f'Figure_1_old_version.{filetype}'), dpi = dpi, format = filetype, 
    bbox_inches='tight', pad_inches=0.1,)       


#### Figure 1 combined (old version, black and white)

In [ ]:
filetype = 'pdf'
dpi = 300

fig, axes = plt.subplots(nrows=4, ncols=1, sharex = 'none', sharey = 'none')
fig.set_size_inches(12,16); fig.set_label('res')
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.15)


# Figure 1A

start_seq = start+30; end_seq = start+90
brs_range = np.arange(start_seq, end_seq+1)
breaths_selected_1 = {key: value for key, value in breaths.items() if key in brs_range}
breaths_selected_1 = pd.concat(breaths_selected_1, sort = True)
breaths_selected_1 = breaths_selected_1.reset_index(level = 0)

breaths_selected_1.pressure.plot(ax = axes[0], color = 'black', title = 'Pressure', linewidth=2, 
    label = 'Pressure',ylim = [0, 17]);
xmin, xmax = axes[0].get_xlim()
axes[0].hlines(10, xmin, xmax, color = 'black', linewidth = 2, linestyle = 'dashed', label = 'PEEP + PS')
axes[0].axvspan(pd.to_datetime('2022-01-25 13:05:33.850'), pd.to_datetime('2022-01-25 13:05:49.250'), 
                                                                     color='gray', alpha=0.4, lw=0)
majorFmt = dates.DateFormatter('%H:%M:%S')  
axes[0].xaxis.set_major_formatter(majorFmt)
plt.setp(axes[0].xaxis.get_majorticklabels(), rotation=0, fontsize = 14, horizontalalignment = 'center')
      
axes[0].set_xlabel('', size = 14, color = 'black', rotation = 0 )
axes[0].set_ylabel('mbar', size = 14, color = 'black')
axes[0].legend(loc=4, ncol=2)
axes[0].set_title('');
axes[0].grid('on', linestyle='-', linewidth=0.5, color = 'gray') 


# Figure 1B-C-D

start_seq = start+60; end_seq = start+80
brs_range = np.arange(start_seq, end_seq+1)
breaths_selected_2 = {key: value for key, value in breaths.items() if key in brs_range}
breaths_selected_2 = pd.concat(breaths_selected_2, sort = True)
breaths_selected_2 = breaths_selected_2.reset_index(level = 0)

breaths_selected_2.pressure.plot(ax = axes[1], color = 'black', title = 'Pressure', linewidth=2, 
    label = 'Pressure',ylim = [0, 17]);
breaths_selected_2.flow.plot(ax = axes[2], color = 'black', title = 'Flow', linewidth=2,
    label = 'Flow', ylim = [-3, 4])
vol_corr = breaths_selected_2.volume / weight
vol_corr.plot(ax = axes[3], color = 'black', title = 'Volume', linewidth=2,
    label = 'Volume', ylim = [0, 9], x_compat=True)

majorFmt = dates.DateFormatter('%H:%M:%S')  
axes[3].xaxis.set_major_formatter(majorFmt)
plt.setp(axes[3].xaxis.get_majorticklabels(), rotation=0, fontsize = 14, horizontalalignment = 'center')

axes[1].text(pd.to_datetime('2022-01-25 13:05:34.200'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:36.170'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:38.330'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:39.860'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:41.300'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:43.400'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:45.000'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:46.400'), 14, '*', fontsize = 18)

xmin, xmax = axes[1].get_xlim()
axes[1].hlines(10, xmin, xmax, color = 'black', linewidth = 2, linestyle = 'dashed', label = 'PEEP + PS')
axes[2].hlines(0, xmin, xmax, color = 'black', linewidth = 2)
axes[3].hlines(4.7, xmin, xmax, color = 'black', linewidth = 2, linestyle = 'dashed', label = 'VTset')
   
axes[1].tick_params(bottom=False, labelbottom=False) 
axes[2].tick_params(bottom=False, labelbottom=False) 
    
axes[1].set_xlabel(''); axes[2].set_xlabel(''); axes[3].set_xlabel('Time', size = 14, color = 'black', rotation = 0 )
axes[1].set_ylabel('mbar', size = 14, color = 'black')
axes[2].set_ylabel('L/min', size = 14, color = 'black')
axes[3].set_ylabel('mL/kg', size = 14, color = 'black')
axes[1].legend(loc=4, ncol=2); axes[1].legend() ; axes[2].legend(ncol=2); axes[3].legend(ncol=2); 
axes[0].set_title(''); axes[1].set_title(''); axes[2].set_title(''); axes[3].set_title('')

fig.text(0.04, 0.90, 'A', fontsize = 16); fig.text(0.04, 0.75, 'B', fontsize = 16)
fig.text(0.04, 0.50, 'C', fontsize = 16); fig.text(0.04, 0.25, 'D', fontsize = 16)
    
fig.savefig(os.path.join(DIR_WRITE, f'Figure_1_old_version_bw.{filetype}'), dpi = dpi, format = filetype, 
    bbox_inches='tight', pad_inches=0.1,)       


#### Figure 1 combined (new version, colored)

In [ ]:
dpi = 300
filetype = 'pdf'

fig = plt.figure(figsize=(12,16))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)

axes = []
axes.append(plt.subplot2grid((4, 8), (0, 0), colspan=8))
axes.append(plt.subplot2grid((4, 8), (1, 1), colspan=7))
axes.append(plt.subplot2grid((4, 8), (2, 1), colspan=7))
axes.append(plt.subplot2grid((4, 8), (3, 1), colspan=7))


# Figure 1A

start_seq = start+30; end_seq = start+90
brs_range = np.arange(start_seq, end_seq+1)
breaths_selected_1 = {key: value for key, value in breaths.items() if key in brs_range}
breaths_selected_1 = pd.concat(breaths_selected_1, sort = True)
breaths_selected_1 = breaths_selected_1.reset_index(level = 0)

breaths_selected_1.pressure.plot(ax = axes[0], color = 'red', title = 'Pressure', linewidth=2, 
    label = 'Pressure',ylim = [0, 17]);
xmin, xmax = axes[0].get_xlim()
axes[0].hlines(10, xmin, xmax, color = 'black', linewidth = 2, linestyle = 'dashed', label = 'PEEP + PS')
axes[0].axvspan(pd.to_datetime('2022-01-25 13:05:33.850'), pd.to_datetime('2022-01-25 13:05:49.250'), 
                                                                     color='gray', alpha=0.4, lw=0)
majorFmt = dates.DateFormatter('%H:%M:%S')  
axes[0].xaxis.set_major_formatter(majorFmt)
plt.setp(axes[0].xaxis.get_majorticklabels(), rotation=0, fontsize = 14, horizontalalignment = 'center')
      
axes[0].set_xlabel('', size = 14, color = 'black', rotation = 0 )
axes[0].set_ylabel('mbar', size = 14, color = 'black')
axes[0].legend(loc=4, ncol=2)
axes[0].set_title('');
axes[0].grid('on', linestyle='-', linewidth=0.5, color = 'gray') 


# Figure 1B-C-D

start_seq = start+60; end_seq = start+80
brs_range = np.arange(start_seq, end_seq+1)
breaths_selected_2 = {key: value for key, value in breaths.items() if key in brs_range}
breaths_selected_2 = pd.concat(breaths_selected_2, sort = True)
breaths_selected_2 = breaths_selected_2.reset_index(level = 0)

breaths_selected_2.pressure.plot(ax = axes[1], color = 'red', title = 'Pressure', linewidth=2, 
    label = 'Pressure',ylim = [0, 17]);
breaths_selected_2.flow.plot(ax = axes[2], color = 'green', title = 'Flow', linewidth=2,
    label = 'Flow', ylim = [-3, 4])
vol_corr = breaths_selected_2.volume / weight
vol_corr.plot(ax = axes[3], color = 'blue', title = 'Volume', linewidth=2,
    label = 'Volume', ylim = [0, 9], x_compat=True)

majorFmt = dates.DateFormatter('%H:%M:%S')  
axes[3].xaxis.set_major_formatter(majorFmt)
plt.setp(axes[3].xaxis.get_majorticklabels(), rotation=0, fontsize = 14, horizontalalignment = 'center')

axes[1].text(pd.to_datetime('2022-01-25 13:05:34.200'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:36.170'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:38.330'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:39.860'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:41.300'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:43.400'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:45.000'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:46.400'), 14, '*', fontsize = 18)

xmin, xmax = axes[1].get_xlim()
axes[1].hlines(10, xmin, xmax, color = 'black', linewidth = 2, linestyle = 'dashed', label = 'PEEP + PS')
axes[2].hlines(0, xmin, xmax, color = 'black', linewidth = 2)
axes[3].hlines(4.7, xmin, xmax, color = 'black', linewidth = 2, linestyle = 'dashed', label = 'VTset')
   
axes[1].tick_params(bottom=False, labelbottom=False) 
axes[2].tick_params(bottom=False, labelbottom=False) 
    
axes[1].set_xlabel(''); axes[2].set_xlabel(''); axes[3].set_xlabel('Time', size = 14, color = 'black', rotation = 0 )
axes[1].set_ylabel('mbar', size = 14, color = 'black')
axes[2].set_ylabel('L/min', size = 14, color = 'black')
axes[3].set_ylabel('mL/kg', size = 14, color = 'black')
axes[1].legend(loc=4, ncol=2); axes[1].legend() ; axes[2].legend(ncol=2); axes[3].legend(ncol=2); 
axes[0].set_title(''); axes[1].set_title(''); axes[2].set_title(''); axes[3].set_title('')

fig.text(0.06, 0.88, 'A', fontsize = 16); fig.text(0.14, 0.70, 'B', fontsize = 16)
fig.text(0.14, 0.52, 'C', fontsize = 16); fig.text(0.14, 0.34, 'D', fontsize = 16)
    
fig.savefig(os.path.join(DIR_WRITE, f'Figure_1.{filetype}'), dpi = dpi, format = filetype, 
    bbox_inches='tight', pad_inches=0.1,)       
    

#### Figure 1 combined (new version, black and white)

In [ ]:
dpi = 300
filetype = 'pdf'

fig = plt.figure(figsize=(12,16))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)

axes = []
axes.append(plt.subplot2grid((4, 8), (0, 0), colspan=8))
axes.append(plt.subplot2grid((4, 8), (1, 1), colspan=7))
axes.append(plt.subplot2grid((4, 8), (2, 1), colspan=7))
axes.append(plt.subplot2grid((4, 8), (3, 1), colspan=7))


# Figure 1A

start_seq = start+30; end_seq = start+90
brs_range = np.arange(start_seq, end_seq+1)
breaths_selected_1 = {key: value for key, value in breaths.items() if key in brs_range}
breaths_selected_1 = pd.concat(breaths_selected_1, sort = True)
breaths_selected_1 = breaths_selected_1.reset_index(level = 0)

breaths_selected_1.pressure.plot(ax = axes[0], color = 'black', title = 'Pressure', linewidth=2, 
    label = 'Pressure',ylim = [0, 17]);
xmin, xmax = axes[0].get_xlim()
axes[0].hlines(10, xmin, xmax, color = 'black', linewidth = 2, linestyle = 'dashed', label = 'PEEP + PS')
axes[0].axvspan(pd.to_datetime('2022-01-25 13:05:33.850'), pd.to_datetime('2022-01-25 13:05:49.250'), 
                                                                     color='gray', alpha=0.4, lw=0)
majorFmt = dates.DateFormatter('%H:%M:%S')  
axes[0].xaxis.set_major_formatter(majorFmt)
plt.setp(axes[0].xaxis.get_majorticklabels(), rotation=0, fontsize = 14, horizontalalignment = 'center')
      
axes[0].set_xlabel('', size = 14, color = 'black', rotation = 0 )
axes[0].set_ylabel('mbar', size = 14, color = 'black')
axes[0].legend(loc=4, ncol=2)
axes[0].set_title('');
axes[0].grid('on', linestyle='-', linewidth=0.5, color = 'gray') 


# Figure 1B-C-D

start_seq = start+60; end_seq = start+80
brs_range = np.arange(start_seq, end_seq+1)
breaths_selected_2 = {key: value for key, value in breaths.items() if key in brs_range}
breaths_selected_2 = pd.concat(breaths_selected_2, sort = True)
breaths_selected_2 = breaths_selected_2.reset_index(level = 0)

breaths_selected_2.pressure.plot(ax = axes[1], color = 'black', title = 'Pressure', linewidth=2, 
    label = 'Pressure',ylim = [0, 17]);
breaths_selected_2.flow.plot(ax = axes[2], color = 'black', title = 'Flow', linewidth=2,
    label = 'Flow', ylim = [-3, 4])
vol_corr = breaths_selected_2.volume / weight
vol_corr.plot(ax = axes[3], color = 'black', title = 'Volume', linewidth=2,
    label = 'Volume', ylim = [0, 9], x_compat=True)

majorFmt = dates.DateFormatter('%H:%M:%S')  
axes[3].xaxis.set_major_formatter(majorFmt)
plt.setp(axes[3].xaxis.get_majorticklabels(), rotation=0, fontsize = 14, horizontalalignment = 'center')

axes[1].text(pd.to_datetime('2022-01-25 13:05:34.200'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:36.170'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:38.330'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:39.860'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:41.300'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:43.400'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:45.000'), 14, '*', fontsize = 18)
axes[1].text(pd.to_datetime('2022-01-25 13:05:46.400'), 14, '*', fontsize = 18)

xmin, xmax = axes[1].get_xlim()
axes[1].hlines(10, xmin, xmax, color = 'black', linewidth = 2, linestyle = 'dashed', label = 'PEEP + PS')
axes[2].hlines(0, xmin, xmax, color = 'black', linewidth = 2)
axes[3].hlines(4.7, xmin, xmax, color = 'black', linewidth = 2, linestyle = 'dashed', label = 'VTset')
   
axes[1].tick_params(bottom=False, labelbottom=False) 
axes[2].tick_params(bottom=False, labelbottom=False) 
    
axes[1].set_xlabel(''); axes[2].set_xlabel(''); axes[3].set_xlabel('Time', size = 14, color = 'black', rotation = 0 )
axes[1].set_ylabel('mbar', size = 14, color = 'black')
axes[2].set_ylabel('L/min', size = 14, color = 'black')
axes[3].set_ylabel('mL/kg', size = 14, color = 'black')
axes[1].legend(loc=4, ncol=2); axes[1].legend() ; axes[2].legend(ncol=2); axes[3].legend(ncol=2); 
axes[0].set_title(''); axes[1].set_title(''); axes[2].set_title(''); axes[3].set_title('')

fig.text(0.06, 0.88, 'A', fontsize = 16); fig.text(0.14, 0.70, 'B', fontsize = 16)
fig.text(0.14, 0.52, 'C', fontsize = 16); fig.text(0.14, 0.34, 'D', fontsize = 16)
    
fig.savefig(os.path.join(DIR_WRITE, f'Figure_1_bw.{filetype}'), dpi = dpi, format = filetype, 
    bbox_inches='tight', pad_inches=0.1,)       
    